Important to clear all outputs before running the project!

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Lucas_Maor_http_input.csv")
df

,msg_id,app_protocol,src_app,dst_app,message,timestamp
0,1,HTTP,client,server,GET /index.html HTTP/1.1,0.010
1,2,HTTP,server,client,HTTP/1.1 200 OK,0.035
2,3,HTTP,client,server,GET /styles.css HTTP/1.1,0.050
3,4,HTTP,server,client,HTTP/1.1 200 OK,0.075
4,5,HTTP,client,server,GET /missing.png HTTP/1.1,0.090
5,6,HTTP,server,client,HTTP/1.1 404 Not Found,0.120
6,7,HTTP,client,server,HEAD /index.html HTTP/1.1,0.140
7,8,HTTP,server,client,HTTP/1.1 200 OK,0.160
8,9,HTTP,client,server,POST /login HTTP/1.1,0.180
9,10,HTTP,server,client,HTTP/1.1 200 OK,0.220


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   msg_id        10 non-null     int64  
 1   app_protocol  10 non-null     object 
 2   src_app       10 non-null     object 
 3   dst_app       10 non-null     object 
 4   message       10 non-null     object 
 5   timestamp     10 non-null     float64
dtypes: float64(1), int64(1), object(4)
memory usage: 612.0+ bytes


In [4]:
df.head()

,msg_id,app_protocol,src_app,dst_app,message,timestamp
0,1,HTTP,client,server,GET /index.html HTTP/1.1,0.010
1,2,HTTP,server,client,HTTP/1.1 200 OK,0.035
2,3,HTTP,client,server,GET /styles.css HTTP/1.1,0.050
3,4,HTTP,server,client,HTTP/1.1 200 OK,0.075
4,5,HTTP,client,server,GET /missing.png HTTP/1.1,0.090


In [5]:
df = df.sort_values("timestamp")
df

,msg_id,app_protocol,src_app,dst_app,message,timestamp
0,1,HTTP,client,server,GET /index.html HTTP/1.1,0.010
1,2,HTTP,server,client,HTTP/1.1 200 OK,0.035
2,3,HTTP,client,server,GET /styles.css HTTP/1.1,0.050
3,4,HTTP,server,client,HTTP/1.1 200 OK,0.075
4,5,HTTP,client,server,GET /missing.png HTTP/1.1,0.090
5,6,HTTP,server,client,HTTP/1.1 404 Not Found,0.120
6,7,HTTP,client,server,HEAD /index.html HTTP/1.1,0.140
7,8,HTTP,server,client,HTTP/1.1 200 OK,0.160
8,9,HTTP,client,server,POST /login HTTP/1.1,0.180
9,10,HTTP,server,client,HTTP/1.1 200 OK,0.220


### Simulating Encapsulation in TCP/IP Layers

This cell simulates the logical encapsulation of an HTTP message into the TCP, IP, and Ethernet layers.
Each layer wraps the data from the upper layer as its payload.

In [1]:
def encapsulate_message(row):
    http = row["message"]

    tcp_segment = {
        "src_port": 12345,
        "dst_port": 80,
        "payload": http
    }

    ip_packet = {
        "src_ip": "192.168.1.10",
        "dst_ip": "192.168.1.20",
        "payload": tcp_segment
    }

    ethernet_frame = {
        "src_mac": "AA:BB:CC:DD:EE:01",
        "dst_mac": "AA:BB:CC:DD:EE:02",
        "payload": ip_packet
    }

    return ethernet_frame

In [7]:
frames = []

for _, row in df.iterrows():
    frame = encapsulate_message(row)
    frames.append(frame)

In [8]:
for _, row in df.iterrows():
    print(
        f"msg_id={row['msg_id']} | "
        f"protocol={row['app_protocol']} | "
        f"{row['src_app']} -> {row['dst_app']} | "
        f"message=\"{row['message']}\" | "
        f"time={row['timestamp']}"
    )

msg_id=1 | protocol=HTTP | client -> server | message="GET /index.html HTTP/1.1" | time=0.01
msg_id=2 | protocol=HTTP | server -> client | message="HTTP/1.1 200 OK" | time=0.035
msg_id=3 | protocol=HTTP | client -> server | message="GET /styles.css HTTP/1.1" | time=0.05
msg_id=4 | protocol=HTTP | server -> client | message="HTTP/1.1 200 OK" | time=0.075
msg_id=5 | protocol=HTTP | client -> server | message="GET /missing.png HTTP/1.1" | time=0.09
msg_id=6 | protocol=HTTP | server -> client | message="HTTP/1.1 404 Not Found" | time=0.12
msg_id=7 | protocol=HTTP | client -> server | message="HEAD /index.html HTTP/1.1" | time=0.14
msg_id=8 | protocol=HTTP | server -> client | message="HTTP/1.1 200 OK" | time=0.16
msg_id=9 | protocol=HTTP | client -> server | message="POST /login HTTP/1.1" | time=0.18
msg_id=10 | protocol=HTTP | server -> client | message="HTTP/1.1 200 OK" | time=0.22


In [9]:
import socket
import threading

# This function starts a simple TCP server that listens for one client connection
def start_server():
    # Create a TCP socket
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Bind the server to localhost and port 8080
    server.bind(("127.0.0.1", 80))

    # Start listening for incoming connections
    server.listen(1)

    # Accept a single client connection
    conn, addr = server.accept()

    # Receive data sent by the client
    data = conn.recv(1024)
    print("Server received:", data.decode(errors="ignore"))

    # Send a basic HTTP response back to the client
    conn.sendall(b"HTTP/1.1 200 OK\r\n\r\n")

    # Close the connection and the server socket
    conn.close()
    server.close()

# Run the server in a separate thread so the notebook does not block
server_thread = threading.Thread(target=start_server)
server_thread.start()

In [10]:
import socket
import time

# Wait briefly to ensure the server is running
time.sleep(1)

# Create a TCP socket for the client
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect to the local server on port 8080
client.connect(("127.0.0.1", 80))

# Send an HTTP GET request to the server
client.sendall(b"GET / HTTP/1.1\r\nHost: localhost\r\n\r\n")
print("Client sent HTTP request")

# Receive the server response
response = client.recv(1024)
print("Client received:", response.decode(errors="ignore"))

# Close the client socket
client.close()


Server received: GET / HTTP/1.1
Host: localhost


Client sent HTTP request
Client received: HTTP/1.1 200 OK


